In [33]:
import requests
import pandas as pd
import numpy as np
from pulp import LpProblem, LpVariable, LpMaximize, lpSum, LpBinary

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

df_players = pd.read_parquet('df_players.parquet')
df_positions = pd.read_parquet('df_positions.parquet')
df_teams = pd.read_parquet('df_teams.parquet')

In [34]:
# Supposons que df_players est votre DataFrame préparé
# Il doit contenir les colonnes : 'position', 'price', 'points_per_cost', 'minutes', 'selected_by_percent'

# Exemple initialisation ou import réel:
# df_players = pd.read_csv('votre_fichier.csv')
# df_players['selected_by_percent'] = df_players['selected_by_percent'].astype(float)

budget = 993  # Exemple budget total (en 0.1M £)
alpha = -0.05    # Pondération ownership (à ajuster: plus grand = plus pénalisant)

df_players = df_players.reset_index(drop=True)  # Indices alignés

# Créer problème d'optimisation
prob = LpProblem("FPL_Team_Optimizer", LpMaximize)

# Variables binaires pour chaque joueur
player_vars = [LpVariable(f"player_{i}", cat=LpBinary) for i in range(len(df_players))]

# Fonction objectif : maximiser points_per_cost tout en minimisant selected_by_percent
prob += lpSum([
    df_players.loc[i, 'points_per_cost'] * player_vars[i] - 
    alpha * df_players.loc[i, 'selected_by_percent'] * player_vars[i]
    for i in range(len(df_players))
])

# Contraintes d’effectif
prob += lpSum(player_vars) == 15
prob += lpSum([player_vars[i] for i in range(len(df_players)) if df_players.loc[i, 'position'] == 'GKP']) == 2
prob += lpSum([player_vars[i] for i in range(len(df_players)) if df_players.loc[i, 'position'] == 'DEF']) == 5
prob += lpSum([player_vars[i] for i in range(len(df_players)) if df_players.loc[i, 'position'] == 'MID']) == 5
prob += lpSum([player_vars[i] for i in range(len(df_players)) if df_players.loc[i, 'position'] == 'FWD']) == 3

# Contrainte : max 3 joueurs par équipe
for club in df_players['team_name'].unique():
    prob += lpSum([
        player_vars[i]
        for i in range(len(df_players))
        if df_players.loc[i, 'team_name'] == club
    ]) <= 3

# Contraintes budget
prob += lpSum([df_players.loc[i, 'price'] * player_vars[i] for i in range(len(df_players))]) <= budget

# Contrainte minimum minutes (exemple)
prob += lpSum([df_players.loc[i, 'minutes'] * player_vars[i] for i in range(len(df_players))]) >= 1000

# Résoudre
prob.solve()

# Extraire liste des joueurs sélectionnés
selected_players = [df_players.loc[i] for i in range(len(df_players)) if player_vars[i].varValue == 1]
selected_df = pd.DataFrame(selected_players)

# Afficher résultats clés
print(selected_df[['web_name', 'position', 'price', 'points_per_cost', 'minutes', 'selected_by_percent']])


Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /usr/local/python/3.12.1/lib/python3.12/site-packages/pulp/apis/../solverdir/cbc/linux/i64/cbc /tmp/1e4fd64cebef4ba796e2c5582918158b-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/1e4fd64cebef4ba796e2c5582918158b-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 32 COLUMNS
At line 5490 RHS
At line 5518 BOUNDS
At line 6259 ENDATA
Problem MODEL has 27 rows, 740 columns and 3362 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 29.3198 - 0.00 seconds
Cgl0003I 0 fixed, 1 tightened bounds, 0 strengthened rows, 0 substitutions
Cgl0004I processed model has 27 rows, 684 columns (684 integer (645 of which binary)) and 3138 elements
Cbc0038I Initial state - 4 integers unsatisfied sum - 0.941176
Cbc0038I Pass   1: suminf.    0.39604 (2) obj. -29.2043 iterations 4
Cbc0038I Pass

In [35]:
selected_df.sort_values(by = ['position'])

,first_name,web_name,team_short,team_name,position,price,total_points,points_per_cost,ict_index,selected_by_percent,selected_rank,value_form,minutes,transfers_in,transfers_in_event,transfers_out,transfers_out_event,expected_goals_per_90,saves_per_90,expected_assists_per_90,expected_goal_involvements_per_90,expected_goals_conceded_per_90,goals_conceded_per_90,defensive_contribution_per_90
0,Marc,Guéhi,CRY,Crystal Palace,DEF,46,29,0.630435,14.9,20.5,19,1.6,360,1156199,117869,252541,5328,0.05,0.00,0.01,0.06,1.00,0.25,6.50
5,Micky,Van de Ven,TOT,Spurs,DEF,46,27,0.586957,14.6,29.4,9,1.5,349,813899,110745,453020,5655,0.18,0.00,0.01,0.19,1.21,0.26,6.19
44,Pedro,Pedro Porro,TOT,Spurs,DEF,56,21,0.375000,20.7,25.3,13,0.9,340,821709,31126,466813,23660,0.03,0.00,0.21,0.24,1.23,0.26,7.15
46,Virgil,Virgil,LIV,Liverpool,DEF,61,22,0.360656,19.2,28.0,11,1.0,360,778923,42657,457075,13829,0.04,0.00,0.02,0.06,0.81,1.00,12.50
71,Maxime,Estève,BUR,Burnley,DEF,40,12,0.300000,11.5,16.7,24,0.8,352,269735,7677,456684,15168,0.01,0.00,0.00,0.01,2.32,1.79,9.72
25,João Pedro,João Pedro,CHE,Chelsea,FWD,77,33,0.428571,33.2,66.2,1,1.1,330,2028863,85109,657946,20264,0.41,0.00,0.05,0.46,0.83,0.55,5.18
87,Hugo,Ekitiké,LIV,Liverpool,FWD,88,24,0.272727,21.7,30.5,8,0.5,299,2071672,40521,675508,76276,0.44,0.00,0.04,0.48,0.84,0.60,3.61
112,Chris,Wood,NFO,Nott'm Forest,FWD,76,18,0.236842,20.4,22.3,16,0.6,296,1831135,34922,1014473,82264,0.49,0.00,0.03,0.52,1.50,1.22,1.22
41,Robert,Sánchez,CHE,Chelsea,GKP,50,19,0.380000,7.9,23.6,14,1.0,360,441195,6678,511348,43058,0.00,2.75,0.00,0.00,0.96,0.75,0.00
59,Martin,Dúbravka,BUR,Burnley,GKP,40,13,0.325000,9.3,34.0,5,0.8,360,483614,19301,257178,10758,0.00,3.00,0.00,0.00,2.29,1.75,0.00


In [36]:
selected_df['price'].sum()

np.int64(991)

In [37]:
selected_df['selected_by_percent'].sum()

np.float64(473.7)